In [1]:
import torch
import torch.nn as nn

18层、34层

In [2]:
class BasicBlock( nn.Module ):
    exp = 1
    def __init__( self, in_channels, out_channels, stride = 1, downsample = None ):
        super( BasicBlock, self ).__init__()
        self.conv1 = nn.Conv2d( in_channels = in_channels,
                                out_channels = out_channels, 
                                kernel_size = 3, 
                                stride = stride, 
                                padding = 1, 
                                bias = False )
        self.bn1 = nn.BatchNorm2d( out_channels )
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d( in_channels = out_channels,
                                out_channels = out_channels,
                                kernel_size = 3,
                                stride = stride,
                                padding = 1,
                                bias = False )
        self.bn2 = nn.BatchNorm2d( out_channels )
        self.downsample = downsample                # 定义下采样方法，默认为None
        
        
        def forward( self, x ):
            idt = x
            if self.downsample is not None:
                idt = self.downsample( x )          # 如果下采样参数不为None就走捷径
            
            x = self.conv1( x )
            x = self.bn1( x )
            x = self.relu( x )
            
            x = self.conv2( x )
            x = self.bn2( x )
            
            x += idt                                # 需要加上捷径输出之后再进入最后一个激活函数
            return self.relu( x )

50层、101层、152层

In [3]:
class Bottleneck( nn.Module ):
    exp = 4                                         # 设置参数来构造第三层
    def __init__( self, in_channels, out_channels, stride = 1, downsample = None ):
        super( Bottleneck, self ).__init__()
        self.conv1 = nn.Conv2d( in_channels = in_channels, 
                                out_channels = out_channels, 
                                kernel_size = 1, 
                                stride = stride, 
                                padding = 1, 
                                bias = False )
        self.bn1 = nn.BatchNorm2d( out_channels )
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d( in_channels = out_channels, 
                                out_channels = out_channels,
                                kernel_size = 3, 
                                stride = stride, 
                                padding = 1, 
                                bias = False )
        self.bn2 = nn.BatchNorm2d( out_channels )
        self.conv3 = nn.Conv2d( in_channels = out_channels, 
                                out_channels = out_channels * self.exp, 
                                kernel_size = 1, 
                                padding = 1, 
                                bias = False )
        self.relu = nn.ReLU( inplace = True )
        self.downsample = downsample
        
    
    def forward( self, x ):
        idt = x
        if self.downsample is not None:
            idt = self.downsample( x )
        
        x = self.conv1( x )
        x = self.bn1( x )
        x = self.relu( x )
        
        x = self.conv2( x )
        x = self.bn2( x )
        x = self.relu( x )
        
        x = self.conv3( x )
        x = self.bn3( x )
        
        x += idt
        return self.relu( x )

![](https://sevanthea7.oss-cn-beijing.aliyuncs.com/QGworks/202407100843088.png)

In [4]:
class ResNet( nn.Module ):
    '''
    block：按照层数传入上面对应的类
    block_num：按照表传入对应列表    18-layers: [ 2, 2, 2, 2 ]   34-layers: [ 3, 4, 6, 3 ], etc
    '''
    def __init__( self, block, block_num, num_classes, include_top = True ):
        super( ResNet, self ).__init__()
        self.include_top = include_top
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d( 3,                      # RGB图像，深度为3
                                self.in_channels, 
                                kernel_size = 7, 
                                stride = 2,
                                padding = 3,
                                bias = False )
        self.bn1 = nn.BatchNorm2d( self.in_channels )   # 卷积核个数变深度
        self.relu = nn.ReLU( inplace = True )
        self.maxpool = nn.MaxPool2d( kernel_size = 3, stride = 2, padding = 1 )
        
        self.layer1 = self._make_layer( block, 64, block_num[0] )
        self.layer2 = self._make_layer( block, 128, block_num[1], stride = 2 )
        self.layer3 = self._make_layer( block, 256, block_num[2], stride = 2 )
        self.layer4 = self._make_layer( block, 512, block_num[3], stride = 2 )
        
        if self.include_top:
            self.avgpool = nn.AdaptiveAvgPool2d( ( 1, 1 ) )                 # 自适应平均池化下采样 -> 出来都是1×1
            self.fc = nn.Linear( 512 * block.exp, num_classes )
            
        for m in self.modules():                                            # 权重初始化
            if isinstance( m, nn.Conv2d ):
                nn.init.kaiming_normal_( m.weight, mode = 'fan_out', nonlinearity = 'relu' )
                
        
        
        
        
    def _make_layer( self, block, channel, block_num, stride = 1 ):
        # 构建捷径函数
        downsample = None
        if stride != 1 or self.in_channels != channel * block.exp:
            downsample = nn.Sequential(
                nn.Conv2d( self.in_channels,
                           channel * block.exp, 
                           kernel_size = 1, 
                           stride = stride, 
                           bias = False ),
                nn.BatchNorm2d( channel * block.exp ),
            )
        layers = []
        layers.append( block( self.in_channels,
                              channel,
                              stride = stride, 
                              downsample = downsample ) )
        self.in_channels = channel * block.exp
        
        for _ in range( 1, block_num ):
            # 捷径在输入和输出形状不一致时通过1x1卷积层调整尺寸以实现加法操作来调整输入的尺寸,第二节开始由于输入和输出形状相同所有都是实线
            layers.append( block( self.in_channels, channel ) )
        
        
        return nn.Sequential( *layers )
    
    
    
    def forward( self, x ):
        x = self.conv1( x )
        x = self.bn1( x )
        x = self.relu1( x )
        x = self.maxpool1( x )
        
        x = self.layer1( x )
        x = self.layer2( x )
        x = self.layer3( x )
        x = self.layer4( x )
        
        if self.include_top:
            x = self.avgpool1( x )
            x = torch.flatten( x, 1 )
            x = self.fc( x )
            
        return x

In [5]:
def ResNet34( num_classes = 5, include_top = True ):
    return ResNet( BasicBlock, [ 3, 4, 6, 3 ], num_classes = num_classes, include_top = include_top )


def ResNet101( num_classes = 5, include_top = True ):
    return ResNet( Bottleneck, [3, 4, 23, 3 ], num_classes = num_classes, include_top = include_top )

In [ ]:
net = ResNet34()
